In [103]:
"""
Lab 4 - COEN 140
Marianne Fuyu Yamazaki Dorr
10/11/2022

This program implements Linear Discrimination Analysis (LDA) and Quadratic Discrimination Analysis (QDA) to analyze 
some data.
"""

import numpy as np
import pandas as pd
import math
import pprint
import time

#turns data from a file into a list
def data_to_list(filename):
    dataList = []
    file = open(filename, 'r')
    
    for line in file:
        stripped_line = line.strip()
        line_list = stripped_line.split(",")
        for x in range(4):
            line_list[x] = np.float64(line_list[x])
            
        dataList.append(line_list)
    
    file.close()
    return dataList   

#split data into training set and testing set (80%/20%)
def split_data(data):
    training_data = []
    testing_data = []
    C1, C2, C3 = ([] for i in range(3))
    index = 0
    #split data into the three categories
    C1name = data[index][4]
    while (data[index][4] == C1name):
        C1.append(data[index])
        index = index + 1
    C2name = data[index][4]

    while (data[index][4] == C2name):
        C2.append(data[index])
        index = index + 1
    C3name = data[index][4]
    while (index < len(data)):
        C3.append(data[index])
        index = index + 1
    #add 80% of each category to training_data list and remaining 20% to testing_data list
    t_index1 = 0.8 * len(C1)
    t_index2 = 0.8 * len(C2)
    t_index3 = 0.8 * len(C3)
    index_category_list = [[t_index1, C1], [t_index2, C2], [t_index3, C3]]
    for c in index_category_list:
        i = 0
        while (i < c[0]):
            training_data.append(c[1][i])
            i += 1
        while (i < len(c[1])):
            testing_data.append(c[1][i])
            i += 1
    return training_data, testing_data

#calculate the means of a matrix
def calc_mean(matrix):
    means = []
    for feature in matrix.T:
        feature = np.array(feature).astype(np.float64)
        means.append(feature.sum()/float(matrix.shape[0]))
    return np.array(means)

#assuming gaussian multivariate, probability function
def pdf(x, mean, covariance):
    s = x.shape[0]
    ratio = 1/math.sqrt(((2*math.pi)**s)* np.linalg.det(covariance))
    exponent = math.exp(-0.5*( np.dot(np.transpose(x-mean), np.dot(np.linalg.inv(covariance),(x-mean)))))
    return ratio*exponent

#linear discrimination analysis function
def LDA(x, means, avg_cov):
    prob = []
    for mean in means: 
        prob.append(pdf(x, mean, avg_cov))
    max_value = max(prob)
    max_index = prob.index(max_value)
    if (max_index == 0):
        return "Iris-setosa"
    if (max_index == 1):
        return "Iris-versicolor"
    if (max_index == 2):
        return "Iris-virginica"
    
def QDA(x, means, covs):
    prob = []
    for i in range(len(means)):
        prob.append(pdf(x, means[i], covs[i]))
    max_value = max(prob)
    max_index = prob.index(max_value)
    if (max_index == 0):
        return "Iris-setosa"
    if (max_index == 1):
        return "Iris-versicolor"
    if (max_index == 2):
        return "Iris-virginica"

#accuracy function
def accuracy(actual, predicted):
    score = 0
    index = -1
    for x in predicted:
        index += 1
        if (actual != x):
            score += 1
            print()
            print("      Error at index: " + str(index))
            print("      Actual Data: " + actual)
            print("      Predicted Data: " + x)
            print()
    return score/len(actual)*100

#http://www.cse.scu.edu/~yfang/coen140/iris.data  
unsplit_data = data_to_list("iris.data.txt")
data = split_data(unsplit_data)
#split data into categories
setosa_train = []
versicolor_train = []
virginica_train = []
for x in data[0]:
    if (x[4] == "Iris-setosa"):
        setosa_train.append(x)
    if (x[4] == "Iris-versicolor"):
        versicolor_train.append(x)
    if (x[4] == "Iris-virginica"):
        virginica_train.append(x)

setosa_test = []
versicolor_test = []
virginica_test = []
for y in data[1]:
    if (y[4] == "Iris-setosa"):
        setosa_test.append(y)
    if (y[4] == "Iris-versicolor"):
        versicolor_test.append(y)
    if (y[4] == "Iris-virginica"):
        virginica_test.append(y)

#convert lists to numpy array
setosa_train = np.array(setosa_train)
versicolor_train = np.array(versicolor_train)
virginica_train = np.array(virginica_train)

setosa_test = np.array(setosa_test)
versicolor_test = np.array(versicolor_test)
virginica_test = np.array(virginica_test)

#strip names from data sets by removing 4th column
setosa_train = np.delete(setosa_train, 4, axis = 1)
versicolor_train = np.delete(versicolor_train, 4, axis = 1)
virginica_train = np.delete(virginica_train, 4, axis = 1)

setosa_test = np.delete(setosa_test, 4, axis = 1)
versicolor_test = np.delete(versicolor_test, 4, axis = 1)
virginica_test = np.delete(virginica_test, 4, axis = 1)

#convert to float64 type
#feature = np.array(feature).astype(np.float64)
setosa_train = np.array(setosa_train).astype(np.float64)
versicolor_train = np.array(versicolor_train).astype(np.float64)
virginica_train = np.array(virginica_train).astype(np.float64)

setosa_test = np.array(setosa_test).astype(np.float64)
versicolor_test = np.array(versicolor_test).astype(np.float64)
virginica_test = np.array(virginica_test).astype(np.float64)

#get the mean of each category using training data set
setosa_mean = calc_mean(setosa_train)
print(setosa_mean)
versicolor_mean = calc_mean(versicolor_train)
virginica_mean = calc_mean(virginica_train)
means = [setosa_mean, versicolor_mean, virginica_mean]

#get the covariance of each category using training data set
setosa_cov = np.cov(setosa_train, y=None, rowvar=False)
print(setosa_cov)
versicolor_cov = np.cov(versicolor_train, y=None, rowvar=False)
virginica_cov = np.cov(virginica_train, y=None, rowvar=False)

covs = [setosa_cov, versicolor_cov, virginica_cov]
#Get the avg covariance for LDA, in order to assume all covariances are the same for each category
avg_cov = (setosa_cov + versicolor_cov + virginica_cov)/3

#Testing LDA on test data set
setosa_pred = []
for x in setosa_test:
    setosa_pred.append(LDA(x, means, avg_cov))
    
versicolor_pred = []
for x in versicolor_test:
    versicolor_pred.append(LDA(x, means, avg_cov))

virginica_pred = []
for x in virginica_test:
    virginica_pred.append(LDA(x, means, avg_cov))

#Testing LDA on training data set
setosa_tpred = []
for x in setosa_train:
    setosa_tpred.append(LDA(x, means, avg_cov))
    
versicolor_tpred = []
for x in versicolor_train:
    versicolor_tpred.append(LDA(x, means, avg_cov))

virginica_tpred = []
for x in virginica_train:
    virginica_tpred.append(LDA(x, means, avg_cov))
    
#find accuracy of LDA
print("--------------------------------------------------------------------------")
print("LDA")
print()
print("________________________")
print("Testing on Setosa data:")
print("________________________")
print()
print("Error rate of LDA on testing data set: " + str(accuracy("Iris-setosa", setosa_pred)) + "%")
print("Error rate of LDA on training data set: " + str(accuracy("Iris-setosa", setosa_tpred)) + "%")
print()
print("________________________")
print("Testing on Versicolor data:")
print("________________________")
print()
print("Error rate of LDA on testing data set: " + str(accuracy("Iris-versicolor", versicolor_pred)) + "%")
print("Error rate of LDA on training data set: " + str(accuracy("Iris-versicolor", versicolor_tpred)) + "%")
print()
print("________________________")
print("Testing on Virginica data:")
print("________________________")
print()
print("Error rate of LDA on testing data set: " + str(accuracy("Iris-virginica", virginica_pred)) + "%")
print("Error rate of LDA on training data set: " + str(accuracy("Iris-virginica", virginica_tpred)) + "%")
print()
print("**************************************************")
print("Combined error rate of training set: " + str((accuracy("Iris-setosa", setosa_tpred)+
                                                     accuracy("Iris-versicolor", versicolor_tpred)
                                                   +accuracy("Iris-virginica", virginica_tpred))/3) + "%")
print()
print("Combined error rate of testing set: " + str((accuracy("Iris-setosa", setosa_pred)+
                                                     accuracy("Iris-versicolor", versicolor_pred)
                                                   +accuracy("Iris-virginica", virginica_pred))/3) + "%")
print("**************************************************")

print("--------------------------------------------------------------------------")
#Testing QDA on test data set
start_time1 = time.time()
qsetosa_pred = []
for x in setosa_test:
    qsetosa_pred.append(QDA(x, means, covs))
    
qversicolor_pred = []
for x in versicolor_test:
    qversicolor_pred.append(QDA(x, means, covs))

qvirginica_pred = []
for x in virginica_test:
    qvirginica_pred.append(QDA(x, means, covs))

#Testing QDA on training data set
qsetosa_tpred = []
for x in setosa_train:
    qsetosa_tpred.append(QDA(x, means, covs))
    
qversicolor_tpred = []
for x in versicolor_train:
    qversicolor_tpred.append(QDA(x, means, covs))

qvirginica_tpred = []
for x in virginica_train:
    qvirginica_tpred.append(QDA(x, means, covs))

end_time1 = time.time()

#find accuracy of QDA
print("--------------------------------------------------------------------------")
print("QDA")
print()
print("________________________")
print("Testing on Setosa data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-setosa", qsetosa_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-setosa", qsetosa_tpred)) + "%")
print()
print("________________________")
print("Testing on Versicolor data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-versicolor", qversicolor_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-versicolor", qversicolor_tpred)) + "%")
print()
print("________________________")
print("Testing on Virginica data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-virginica", qvirginica_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-virginica", qvirginica_tpred)) + "%")
print()
print("**************************************************")
print("Combined error rate of training set: " + str((accuracy("Iris-setosa", qsetosa_tpred)+
                                                     accuracy("Iris-versicolor", qversicolor_tpred)
                                                   +accuracy("Iris-virginica", qvirginica_tpred))/3) + "%")
print()
print("Combined error rate of testing set: " + str((accuracy("Iris-setosa", qsetosa_pred)+
                                                     accuracy("Iris-versicolor", qversicolor_pred)
                                                   +accuracy("Iris-virginica", qvirginica_pred))/3) + "%")
print("**************************************************")
print()
print("Time of QDA " + str(end_time1 - start_time1))
print("--------------------------------------------------------------------------")

"""
PART 5 - Diagonal Matrix
"""
diag_setosa_cov = np.zeros((4,4))
diag_versicolor_cov = np.zeros((4,4))
diag_virginica_cov = np.zeros((4,4))

for i in range(4):
    diag_setosa_cov[i][i] = setosa_cov[i][i]

for i in range(4):
    diag_versicolor_cov[i][i] = versicolor_cov[i][i]

for i in range(4):
    diag_virginica_cov[i][i] = virginica_cov[i][i]

diag_covs = [diag_setosa_cov, diag_versicolor_cov, diag_virginica_cov]

#Testing QDA on test data set
start_time2 = time.time()
qsetosa_pred = []
for x in setosa_test:
    qsetosa_pred.append(QDA(x, means, diag_covs))
    
qversicolor_pred = []
for x in versicolor_test:
    qversicolor_pred.append(QDA(x, means, diag_covs))

qvirginica_pred = []
for x in virginica_test:
    qvirginica_pred.append(QDA(x, means, diag_covs))

#Testing QDA on training data set
qsetosa_tpred = []
for x in setosa_train:
    qsetosa_tpred.append(QDA(x, means, diag_covs))
    
qversicolor_tpred = []
for x in versicolor_train:
    qversicolor_tpred.append(QDA(x, means, diag_covs))

qvirginica_tpred = []
for x in virginica_train:
    qvirginica_tpred.append(QDA(x, means, diag_covs))

end_time2 = time.time()
#find accuracy of QDA
print("--------------------------------------------------------------------------")
print("QDA - Diagonal Covariance Analysis")
print()
print("________________________")
print("Testing on Setosa data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-setosa", qsetosa_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-setosa", qsetosa_tpred)) + "%")
print()
print("________________________")
print("Testing on Versicolor data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-versicolor", qversicolor_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-versicolor", qversicolor_tpred)) + "%")
print()
print("________________________")
print("Testing on Virginica data:")
print("________________________")
print()
print("Error rate of QDA on testing data set: " + str(accuracy("Iris-virginica", qvirginica_pred)) + "%")
print("Error rate of QDA on training data set: " + str(accuracy("Iris-virginica", qvirginica_tpred)) + "%")
print()
print("**************************************************")
print("Combined error rate of training set: " + str((accuracy("Iris-setosa", qsetosa_tpred)+
                                                     accuracy("Iris-versicolor", qversicolor_tpred)
                                                   +accuracy("Iris-virginica", qvirginica_tpred))/3) + "%")
print()
print("Combined error rate of testing set: " + str((accuracy("Iris-setosa", qsetosa_pred)+
                                                     accuracy("Iris-versicolor", qversicolor_pred)
                                                   +accuracy("Iris-virginica", qvirginica_pred))/3) + "%")
print("**************************************************")
print()
print("Time of Diagonal QDA " + str(end_time2 - start_time2))
print("--------------------------------------------------------------------------")


    


[5.0375 3.44   1.4625 0.2325]
[[0.13112179 0.09897436 0.01298077 0.01362179]
 [0.09897436 0.13271795 0.00205128 0.0145641 ]
 [0.01298077 0.00205128 0.02958333 0.00458333]
 [0.01362179 0.0145641  0.00458333 0.00994231]]
--------------------------------------------------------------------------
LDA

________________________
Testing on Setosa data:
________________________

Error rate of LDA on testing data set: 0.0%
Error rate of LDA on training data set: 0.0%

________________________
Testing on Versicolor data:
________________________

Error rate of LDA on testing data set: 0.0%

      Error at index: 20
      Actual Data: Iris-versicolor
      Predicted Data: Iris-virginica


      Error at index: 33
      Actual Data: Iris-versicolor
      Predicted Data: Iris-virginica

Error rate of LDA on training data set: 13.333333333333334%

________________________
Testing on Virginica data:
________________________

Error rate of LDA on testing data set: 0.0%

      Error at index: 33
      